In [6]:
pip install pygame


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pygame
import math
import random
import numpy as np

pygame.init()

########################################################
# Settings
########################################################
EVASION_THRESHOLD = 40
w, h = 800, 600
evader_speed = 8
pursuer_speed = 1
evader_radius, pursuer_radius, goal_radius = 15, 15, 20
max_goals = 10

# Colors
WHITE, BLUE, RED, GREEN, BLACK = (255,255,255), (50,150,255), (255,80,80), (50,200,50), (0,0,0)

# Initial positions
evader_x =  w//4
evader_y =  h//2
pursuer_x = 3*w//4 
pursuer_y = h//2
goal_counter = 0
goal_times = []

#Set up game
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Pursuer-Evader Game")
clock = pygame.time.Clock()

# Initialize first goal
goal_x, goal_y = random.randint(0, w), random.randint(0, h)
goal_spawn_time = pygame.time.get_ticks() / 1000.0
running = True
game_over = False


while running:
    #set the speed of the game
    clock.tick(120)
    screen.fill(WHITE)

    if not game_over:
        ########################################################
        # MOVEEEE
        ########################################################
        #caluculate hte current distance between the pursuer and the evader
        pursuer_dx = pursuer_x - evader_x
        pursuer_dy = pursuer_y - evader_y
        dist_to_pursuer = np.sqrt(pursuer_dx**2+ pursuer_dy**2)

        # engage in straight line movement towards goal or evasion
        if dist_to_pursuer >= EVASION_THRESHOLD:
            # Go to goal via a straight line
            goal_dx = goal_x - evader_x
            goal_dy = goal_y - evader_y
            goal_dist = np.sqrt(goal_dx**2+ goal_dy**2)
            #if the distance is greater than 0, move the evader towards the goal
            if goal_dist > 0:
                evader_x += (goal_dx / goal_dist) * evader_speed
                evader_y += (goal_dy / goal_dist) * evader_speed
        else:
            # Evasion mode: move away from pursuer
            # Calculate direction away from pursuer (evader to pursuer reversed)
            evasion_dx = evader_x - pursuer_x
            evasion_dy = evader_y - pursuer_y
            evasion_dist = np.sqrt(evasion_dx**2 + evasion_dy**2)
            if evasion_dist > 0:
                # Normalize and move away from pursuer
                evader_x += (evasion_dx / evasion_dist) * evader_speed
                evader_y += (evasion_dy / evasion_dist) * evader_speed
       
        # update the position of the evader
        evader_x = max(evader_radius, min(w - evader_radius, evader_x))
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        # Pursuer chases evader
        capture_distance_dx = evader_x - pursuer_x
        capture_distance_dy = evader_y - pursuer_y
        capture_distance = np.sqrt(capture_distance_dx**2 + capture_distance_dy**2)
        if capture_distance > 0:
            pursuer_x += (capture_distance_dx / capture_distance) * pursuer_speed
            pursuer_y += (capture_distance_dy / capture_distance) * pursuer_speed

        # update the position of the pursuer
        pursuer_x = max(pursuer_radius, min(w - pursuer_radius, pursuer_x))
        pursuer_y = max(pursuer_radius, min(h - pursuer_radius, pursuer_y))

        ########################################################
        # CHECK COLLISIONS WITH ADVERSARY AND GOAL
        ########################################################
        #current distance between the pursuer and the evader
        capture_distance_dx_postupdate = evader_x - pursuer_x
        capture_distance_dy_postupdate = evader_y - pursuer_y
        capture_distance_post_update = np.sqrt(capture_distance_dx_postupdate**2 + capture_distance_dy_postupdate**2)
        if capture_distance_post_update <= evader_radius + pursuer_radius:
            game_over = True
            running = False

        # Check goal collection
        goal_distance_dx_postupdate = evader_x - goal_x
        goal_distance_dy_postupdate = evader_y - goal_y
        goal_distance_post_update = np.sqrt(goal_distance_dx_postupdate**2 + goal_distance_dy_postupdate**2)
        if goal_counter < max_goals and goal_distance_post_update <= evader_radius + goal_radius:
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time)
            goal_counter += 1
            if goal_counter < max_goals:
                goal_x, goal_y = random.randint(0, w), random.randint(0, h)
                goal_spawn_time = pygame.time.get_ticks() / 1000.0
            else:
                game_over = True
                running = False

    ########################################################
    # Draw circles
    ########################################################
    pygame.draw.circle(screen, RED, (int(pursuer_x), int(pursuer_y)), pursuer_radius)
    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    #draw the line between the pursuer and the evader for visualization
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer_x), int(pursuer_y)), 1)
    if goal_counter < max_goals:
        pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)
    

    # Display
    font = pygame.font.Font(None, 36)
    if not game_over:
        screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 10))
        dist_to_pursuer = math.hypot(pursuer_x - evader_x, pursuer_y - evader_y)
        if goal_counter < max_goals and dist_to_pursuer >= EVASION_THRESHOLD:
            screen.blit(font.render("Mode: GOAL-SEEKING", True, GREEN), (10, 50))
        else:
            screen.blit(font.render("Mode: EVASION", True, RED), (10, 50))
        screen.blit(font.render("C: close", True, BLACK), (10, 90))
    else:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals collected!", True, GREEN)
            screen.blit(msg, msg.get_rect(center=(w//2, h//2)))
        else:
            msg = font.render("GAME OVER - Caught!", True, RED)
            screen.blit(msg, msg.get_rect(center=(w//2, h//2)))

    pygame.display.flip()

pygame.quit()



